# Unified Catalog: Hierarchical Terms Management

This notebook demonstrates how to create and manage hierarchical term structures in Microsoft Purview Unified Catalog using the `parent-id` feature.

## Prerequisites

- Purview CLI installed (`pip install purviewcli`)
- Authenticated to your Purview account
- A governance domain created
- Environment variables set:
  - `PURVIEW_ACCOUNT_NAME`
  - `AZURE_TENANT_ID`, `AZURE_CLIENT_ID`, `AZURE_CLIENT_SECRET` (or interactive auth)

## Overview

Hierarchical terms allow you to organize glossary terms in parent-child relationships, creating taxonomies like:

```
Data Quality (Root)
├── Accuracy
│   ├── Field Accuracy
│   └── Record Accuracy
├── Completeness
└── Consistency
```

In [ ]:
# Import required libraries
import subprocess
import json
import os
from typing import Dict, List, Optional

# Helper function to run pvw commands
def run_pvw_command(command: List[str], capture_output=True) -> Dict:
    """Execute a pvw CLI command and return JSON output."""
    try:
        result = subprocess.run(
            command,
            capture_output=capture_output,
            text=True,
            check=False
        )
        
        if result.returncode != 0:
            print(f"Error: {result.stderr}")
            return {"error": result.stderr}
        
        # Try to parse JSON output
        try:
            return json.loads(result.stdout)
        except json.JSONDecodeError:
            return {"output": result.stdout}
    except Exception as e:
        print(f"Exception: {str(e)}")
        return {"error": str(e)}

print("✅ Helper functions loaded")

## Step 1: Configuration

Set up your domain ID and other configuration parameters.

In [ ]:
# Configuration
DOMAIN_ID = "your-domain-guid-here"  # Replace with your governance domain ID
PURVIEW_ACCOUNT = os.getenv("PURVIEW_ACCOUNT_NAME", "your-purview-account")

# Verify configuration
print(f"Purview Account: {PURVIEW_ACCOUNT}")
print(f"Domain ID: {DOMAIN_ID}")
print("\n⚠️  Make sure to replace 'your-domain-guid-here' with your actual domain GUID")

## Step 2: Create Root Term

First, we'll create a root-level term that will serve as the parent for other terms.

In [ ]:
# Create root term - Data Quality Framework
root_term_command = [
    "pvw", "uc", "term", "create",
    "--name", "Data Quality Framework",
    "--description", "Comprehensive framework for measuring data quality across all dimensions",
    "--domain-id", DOMAIN_ID,
    "--status", "Published",
    "--acronym", "DQF"
]

print("Creating root term: Data Quality Framework...")
root_result = run_pvw_command(root_term_command)

if "error" not in root_result:
    ROOT_TERM_ID = root_result.get("id") or root_result.get("guid")
    print(f"✅ Root term created successfully!")
    print(f"   Term ID: {ROOT_TERM_ID}")
    print(f"   Name: {root_result.get('name')}")
else:
    print(f"❌ Error creating root term: {root_result.get('error')}")
    ROOT_TERM_ID = None

## Step 3: Create Child Terms (Level 1)

Now we'll create first-level child terms under the root term.

In [ ]:
# Create child terms representing data quality dimensions
if ROOT_TERM_ID:
    dimensions = [
        {
            "name": "Accuracy",
            "description": "Measures the degree to which data correctly represents the real-world entity or event",
            "acronym": "ACC"
        },
        {
            "name": "Completeness",
            "description": "Measures whether all required data is present",
            "acronym": "COMP"
        },
        {
            "name": "Consistency",
            "description": "Measures whether data is uniform across all systems and over time",
            "acronym": "CONS"
        },
        {
            "name": "Timeliness",
            "description": "Measures whether data is available when needed",
            "acronym": "TIME"
        }
    ]
    
    dimension_ids = {}
    
    for dim in dimensions:
        print(f"\nCreating dimension: {dim['name']}...")
        
        child_command = [
            "pvw", "uc", "term", "create",
            "--name", dim['name'],
            "--description", dim['description'],
            "--domain-id", DOMAIN_ID,
            "--parent-id", ROOT_TERM_ID,  # ← Setting parent relationship
            "--status", "Published",
            "--acronym", dim['acronym']
        ]
        
        result = run_pvw_command(child_command)
        
        if "error" not in result:
            term_id = result.get("id") or result.get("guid")
            dimension_ids[dim['name']] = term_id
            print(f"✅ Created: {dim['name']} (ID: {term_id})")
        else:
            print(f"❌ Error: {result.get('error')}")
    
    print(f"\n📊 Created {len(dimension_ids)} dimension terms")
else:
    print("⚠️  Skipping child term creation - root term not created")

## Step 4: Create Grandchild Terms (Level 2)

Let's create specific metrics under the Accuracy dimension.

In [ ]:
# Create second-level terms (grandchildren of root)
if "Accuracy" in dimension_ids:
    ACCURACY_ID = dimension_ids["Accuracy"]
    
    accuracy_metrics = [
        {
            "name": "Field-Level Accuracy",
            "description": "Accuracy measured at individual field/column level",
            "status": "Draft"
        },
        {
            "name": "Record-Level Accuracy",
            "description": "Accuracy measured at entire record level",
            "status": "Draft"
        },
        {
            "name": "Syntactic Accuracy",
            "description": "Correctness of data format and structure",
            "status": "Draft"
        },
        {
            "name": "Semantic Accuracy",
            "description": "Correctness of data meaning and context",
            "status": "Draft"
        }
    ]
    
    print(f"Creating metrics under 'Accuracy' dimension...")
    
    for metric in accuracy_metrics:
        grandchild_command = [
            "pvw", "uc", "term", "create",
            "--name", metric['name'],
            "--description", metric['description'],
            "--domain-id", DOMAIN_ID,
            "--parent-id", ACCURACY_ID,  # ← Parent is the Accuracy dimension
            "--status", metric['status']
        ]
        
        result = run_pvw_command(grandchild_command)
        
        if "error" not in result:
            print(f"✅ Created: {metric['name']}")
        else:
            print(f"❌ Error: {result.get('error')}")
else:
    print("⚠️  Skipping grandchild term creation - Accuracy dimension not created")

## Step 5: Update Existing Terms to Add Parent

You can also update existing terms to add them to a hierarchy.

In [ ]:
# Example: Update an existing term to set its parent
# First, let's create a standalone term
standalone_command = [
    "pvw", "uc", "term", "create",
    "--name", "Data Validity",
    "--description", "Term created without parent initially",
    "--domain-id", DOMAIN_ID,
    "--status", "Draft"
]

print("Creating standalone term...")
standalone_result = run_pvw_command(standalone_command)

if "error" not in standalone_result:
    standalone_id = standalone_result.get("id") or standalone_result.get("guid")
    print(f"✅ Created standalone term: {standalone_id}")
    
    # Now update it to add parent relationship
    if ROOT_TERM_ID:
        print(f"\nUpdating term to add parent relationship...")
        update_command = [
            "pvw", "uc", "term", "update",
            "--term-id", standalone_id,
            "--parent-id", ROOT_TERM_ID  # ← Adding parent to existing term
        ]
        
        update_result = run_pvw_command(update_command)
        
        if "error" not in update_result:
            print(f"✅ Successfully added parent to 'Data Validity' term")
        else:
            print(f"❌ Error updating term: {update_result.get('error')}")
else:
    print(f"❌ Error creating standalone term: {standalone_result.get('error')}")

## Step 6: Bulk Update from CSV

For large-scale operations, use CSV files to manage hierarchical terms.

In [ ]:
# Create a CSV file for bulk updates
import csv
from io import StringIO

csv_content = """term_id,name,description,parent_id,status,acronyms
{root_id},Data Quality Framework,Root category for data quality,,Published,DQF
{comp_id},Completeness,Data completeness dimension,{root_id},Published,COMP
{cons_id},Consistency,Data consistency dimension,{root_id},Published,CONS
new-term-1,Uniqueness,Uniqueness of data records,{root_id},Draft,UNIQ
""".format(
    root_id=ROOT_TERM_ID if ROOT_TERM_ID else "root-guid",
    comp_id=dimension_ids.get("Completeness", "comp-guid"),
    cons_id=dimension_ids.get("Consistency", "cons-guid")
)

# Save to file
csv_file_path = "hierarchical_terms_update.csv"
with open(csv_file_path, 'w', newline='') as f:
    f.write(csv_content)

print(f"✅ Created CSV file: {csv_file_path}")
print("\nCSV Content:")
print(csv_content)

# Execute bulk update (dry-run first)
print("\n" + "="*60)
print("Running DRY RUN to preview changes...")
dry_run_command = [
    "pvw", "uc", "term", "update-csv",
    "--csv-file", csv_file_path,
    "--dry-run"
]

run_pvw_command(dry_run_command, capture_output=False)

print("\n💡 To apply changes, run without --dry-run flag")

## Step 7: Query and Verify Hierarchy

Let's retrieve and verify the hierarchical structure we created.

In [ ]:
# Get all terms in the domain
list_command = [
    "pvw", "uc", "term", "list",
    "--domain-id", DOMAIN_ID
]

print("Retrieving all terms in domain...")
terms_result = run_pvw_command(list_command)

if "error" not in terms_result:
    terms = terms_result.get("value", [])
    print(f"✅ Found {len(terms)} terms in domain\n")
    
    # Build hierarchy map
    hierarchy = {}
    for term in terms:
        term_id = term.get("id") or term.get("guid")
        parent_id = term.get("parentId")
        name = term.get("name")
        status = term.get("status")
        
        if not parent_id:
            # Root level term
            print(f"📌 {name} (Root)")
            print(f"   Status: {status}, ID: {term_id}")
        else:
            # Child term - store for later display
            if parent_id not in hierarchy:
                hierarchy[parent_id] = []
            hierarchy[parent_id].append({
                "id": term_id,
                "name": name,
                "status": status
            })
    
    # Display children
    if ROOT_TERM_ID and ROOT_TERM_ID in hierarchy:
        print(f"\n📂 Children of 'Data Quality Framework':")
        for child in hierarchy[ROOT_TERM_ID]:
            print(f"   ├── {child['name']} ({child['status']})")
            
            # Display grandchildren if any
            if child['id'] in hierarchy:
                for grandchild in hierarchy[child['id']]:
                    print(f"   │   └── {grandchild['name']} ({grandchild['status']})")
else:
    print(f"❌ Error retrieving terms: {terms_result.get('error')}")

## Step 8: Moving Terms to Different Parents

Demonstrate how to reorganize hierarchy by changing parent relationships.

In [ ]:
# Example: Move a term from one parent to another
# First, create a new parent category
new_parent_command = [
    "pvw", "uc", "term", "create",
    "--name", "Advanced Data Quality Metrics",
    "--description", "Advanced and specialized data quality metrics",
    "--domain-id", DOMAIN_ID,
    "--status", "Published"
]

print("Creating new parent category...")
new_parent_result = run_pvw_command(new_parent_command)

if "error" not in new_parent_result:
    new_parent_id = new_parent_result.get("id") or new_parent_result.get("guid")
    print(f"✅ Created new parent: {new_parent_id}")
    
    # Now let's move a term to this new parent
    # (We'll use one of the accuracy metrics if it exists)
    print("\n💡 Example: To move a term to this new parent:")
    print(f"   pvw uc term update --term-id <term-guid> --parent-id {new_parent_id}")
else:
    print(f"❌ Error: {new_parent_result.get('error')}")

## Step 9: Remove Parent (Make Term Top-Level)

Show how to remove parent relationship and make a term top-level.

In [ ]:
# Example: Remove parent to make a term top-level
print("Example: Removing parent relationship")
print("Command: pvw uc term update --term-id <term-guid> --parent-id \"\"")
print("\n💡 Setting parent-id to empty string removes the parent relationship")
print("   The term becomes a top-level term with no parent")

## Step 10: Best Practices Summary

### Key Takeaways

1. **Create Top-Down**: Always create parent terms before child terms
2. **Use Meaningful Names**: Make the hierarchy intuitive
3. **Limit Depth**: Keep hierarchies 3-5 levels deep
4. **Publish Parents First**: Ensure parent terms are published before referencing
5. **Use Bulk Operations**: For large-scale changes, use CSV/JSON bulk updates
6. **Test with Dry-Run**: Always preview bulk changes before applying
7. **Document Structure**: Maintain documentation of your taxonomy

### Common Patterns

- **Quality Dimensions**: Root → Dimension → Metric → Sub-Metric
- **Business Domains**: Company → Division → Department → Function
- **Data Types**: Root Type → Category → Specific Type → Variant
- **Geography**: Region → Country → State → City

### Validation Checklist

- ✅ Parent term exists before creating children
- ✅ All terms are in the same governance domain
- ✅ No circular references (term cannot be its own ancestor)
- ✅ GUIDs are properly formatted
- ✅ Preview changes with dry-run before bulk updates

## Resources

- [Hierarchical Terms Documentation](../../../doc/commands/unified-catalog/hierarchical-terms.md)
- [CSV Sample File](../csv/uc_term_update_sample.csv)
- [JSON Sample File](../json/uc_term_update_sample.json)
- [PowerShell Examples](../powershell/uc_parent_id_examples.ps1)

## Next Steps

1. Design your term taxonomy
2. Create CSV files with your hierarchy
3. Test with a few terms first
4. Scale to full taxonomy
5. Monitor and maintain over time

In [ ]:
print("✅ Notebook execution complete!")
print("\n📚 Review the hierarchy in your Purview portal:")
print(f"   https://{PURVIEW_ACCOUNT}.purview.azure.com")